# 🛒 Univerzální E-shop Scraper

Nástroj pro stahování produktových dat z e-shopů do Excelu.

## Návod:
1. **Buňka 1** - Instalace (spusťte jednou)
2. **Buňka 2** - Zadejte URL webu
3. **Buňka 3** - Spusťte scraping
4. **Buňka 4** - Stáhněte Excel
5. **Buňka 5** - Reset pro nový web

💡 **Tip:** Můžete kdykoliv zastavit (⏹️) a stáhnout částečné výsledky.

In [ ]:
#@title 📦 Buňka 1: Instalace
!pip install requests beautifulsoup4 pandas openpyxl lxml -q
print("✅ Instalace dokončena")

In [ ]:
#@title 🌐 Buňka 2: Nastavení webu
URL_WEBU = "https://aktin.cz" #@param {type:"string"}

# Vyčistit URL
URL_WEBU = URL_WEBU.strip().rstrip('/')
print(f"🎯 Cílový web: {URL_WEBU}")

In [ ]:
#@title 🚀 Buňka 3: Scraping (lze zastavit)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import json
import random
from urllib.parse import urljoin, urlparse
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# GLOBÁLNÍ - přežijí zastavení
if 'products_data' not in dir(): products_data = []
if 'all_product_urls' not in dir(): all_product_urls = set()
if 'processed_urls' not in dir(): processed_urls = set()
if 'visited_pages' not in dir(): visited_pages = set()

BASE_URL = URL_WEBU
DOMAIN = urlparse(BASE_URL).netloc

USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120.0.0.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
]

HEADERS = {
    'User-Agent': random.choice(USER_AGENTS),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'cs-CZ,cs;q=0.9,en;q=0.8',
}

KNOWN_CATEGORIES = {
    'aktin.cz': ['/proteiny', '/aminokyseliny', '/kreatin', '/gainery', '/spalovace-tuku',
                 '/vitaminy-mineraly', '/zdravi', '/superfood', '/orechova-masla', '/snacky',
                 '/napoje', '/potraviny', '/tycinky', '/pomucky', '/obleceni', '/novinky'],
    'brainmarket.cz': ['/brainmax-doplnky-stravy/', '/brainmax-pure/', '/doplnky-stravy/',
                       '/potraviny-bm/', '/kosmetika-drogerie/', '/novinky/'],
}

session = requests.Session()
session.headers.update(HEADERS)

def get_page(url, retries=3):
    for i in range(retries):
        try:
            session.headers['User-Agent'] = random.choice(USER_AGENTS)
            r = session.get(url, timeout=30)
            if r.status_code == 200: return r.text
            time.sleep(3)
        except: time.sleep(3)
    return None

def clean_price(t):
    if not t: return ""
    p = re.sub(r'[^\d,.]', '', str(t)).replace(',', '.')
    return p

def clean_text(t):
    if not isinstance(t, str): return str(t) if t else ""
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', t).strip()

def is_product_url(url):
    if not url or DOMAIN not in url: return False
    excluded = ['/kosik', '/cart', '/login', '/registrace', '/kontakt', '/blog',
                '/podminky', '/gdpr', '/sitemap', '/search', '/kategorie', '/strana-',
                '.pdf', '.jpg', '.png', '/admin', '/api']
    path = urlparse(url).path.lower()
    for e in excluded:
        if e in path: return False
    return len(path) > 5

def find_links(soup, base):
    urls = set()
    selectors = ['a.p-name', '.product a', '.products a', 'h2 a', 'h3 a', '.item a', '.card a']
    for sel in selectors:
        for a in soup.select(sel):
            href = a.get('href', '')
            if href:
                full = urljoin(base, href).split('#')[0]
                if is_product_url(full): urls.add(full)
    return urls

def extract_data(url):
    html = get_page(url)
    if not html: return None
    soup = BeautifulSoup(html, 'html.parser')
    data = {'nazev': '', 'ean': '', 'cena': '', 'cena_puvodni': '', 'sleva': '', 'dostupnost': '', 'url': url}
    
    # Název
    for sel in ['h1', '[itemprop="name"]', '.product-title']:
        el = soup.select_one(sel)
        if el:
            data['nazev'] = clean_text(el.get_text(strip=True))
            if data['nazev']: break
    if not data['nazev']: return None
    
    # EAN
    for script in soup.select('script[type="application/ld+json"]'):
        try:
            j = json.loads(script.string or '{}')
            m = re.search(r'"gtin13?"\s*:\s*"?(\d{8,14})"?', json.dumps(j))
            if m: data['ean'] = m.group(1); break
        except: pass
    if not data['ean']:
        m = re.search(r'EAN[:\s]*(\d{8,14})', html, re.I)
        if m: data['ean'] = m.group(1)
    
    # Cena
    for sel in ['[itemprop="price"]', '.price-final', '.p-final', '.price', '.current-price']:
        el = soup.select_one(sel)
        if el:
            data['cena'] = clean_price(el.get('content') or el.get_text())
            if data['cena']: break
    
    # Původní cena
    for sel in ['.price-standard', '.p-standard', '.old-price', 'del']:
        el = soup.select_one(sel)
        if el:
            data['cena_puvodni'] = clean_price(el.get_text())
            if data['cena_puvodni']: break
    
    # Sleva
    if data['cena'] and data['cena_puvodni']:
        try:
            c, o = float(data['cena']), float(data['cena_puvodni'])
            if o > c: data['sleva'] = f"{((o-c)/o)*100:.0f}%"
        except: pass
    
    # Dostupnost
    el = soup.select_one('.availability, .p-availability, .stock')
    if el: data['dostupnost'] = clean_text(el.get_text(strip=True))
    
    return data

print("="*60)
print(f"🛒 E-SHOP SCRAPER")
print(f"🎯 Web: {BASE_URL}")
print(f"📊 Staženo: {len(products_data)} | URL: {len(all_product_urls)}")
print("="*60)
print("💡 Stop = ⏹️ | Pak Buňka 4 pro stažení")
print("="*60)

try:
    # Fáze 1: Sbírání URL
    if not all_product_urls:
        print("\n📁 Fáze 1: Hledání produktů\n")
        to_visit = {BASE_URL}
        for d, cats in KNOWN_CATEGORIES.items():
            if d in DOMAIN:
                to_visit.update([BASE_URL + c for c in cats])
        
        while to_visit and len(visited_pages) < 500:
            url = to_visit.pop()
            if url in visited_pages: continue
            print(f"   [{len(visited_pages)+1}] {url[:55]}...", end=" ", flush=True)
            html = get_page(url)
            if not html: print("❌"); visited_pages.add(url); continue
            soup = BeautifulSoup(html, 'html.parser')
            visited_pages.add(url)
            new = find_links(soup, url)
            before = len(all_product_urls)
            all_product_urls.update(new)
            print(f"✅ +{len(all_product_urls)-before} ({len(all_product_urls)})")
            # Pagination
            for a in soup.select('a.next, a[rel="next"], .pagination a'):
                href = a.get('href')
                if href:
                    full = urljoin(url, href)
                    if DOMAIN in full and full not in visited_pages:
                        to_visit.add(full)
            time.sleep(random.uniform(0.5, 1.5))
        print(f"\n📊 Nalezeno {len(all_product_urls)} URL")
    
    # Fáze 2: Stahování
    print("\n📦 Fáze 2: Stahování detailů\n")
    todo = list(all_product_urls - processed_urls)
    total = len(todo)
    for i, url in enumerate(todo, 1):
        print(f"\r   [{i}/{total}] {(i/total)*100:.1f}% | Produktů: {len(products_data)}", end="", flush=True)
        data = extract_data(url)
        if data: products_data.append(data)
        processed_urls.add(url)
        time.sleep(random.uniform(0.5, 1.2))
        if i % 50 == 0:
            pd.DataFrame(products_data).to_excel('/content/temp.xlsx', index=False)

except KeyboardInterrupt:
    print("\n\n⏹️ ZASTAVENO")

print(f"\n\n{'='*60}")
print(f"✅ Staženo: {len(products_data)} produktů")
print(f"   S EAN: {len([p for p in products_data if p['ean']])}")
print(f"   Zbývá: {len(all_product_urls) - len(processed_urls)}")
print("="*60)

In [ ]:
#@title 💾 Buňka 4: Stažení Excel
from google.colab import files
from datetime import datetime
import re

def clean(t):
    if not isinstance(t, str): return str(t) if t else ""
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', t).strip()

if 'products_data' in dir() and products_data:
    data = [{k: clean(v) for k, v in p.items()} for p in products_data]
    df = pd.DataFrame(data)
    df.columns = ['Název produktu', 'EAN', 'Cena', 'Původní cena', 'Sleva', 'Dostupnost', 'URL']
    df = df.drop_duplicates(subset=['Název produktu', 'URL']).sort_values('Název produktu')
    
    domain = DOMAIN.replace('www.', '').replace('.', '_')
    fn = f'{domain}_{datetime.now().strftime("%Y%m%d_%H%M")}.xlsx'
    
    print("="*60)
    print(f"📊 Export: {len(df)} produktů")
    print(f"   S EAN: {len(df[df['EAN'].str.len() > 0])}")
    print("="*60)
    
    df.to_excel(fn, index=False)
    files.download(fn)
else:
    print("❌ Žádná data")

In [ ]:
#@title 🔄 Buňka 5: Reset pro nový web
products_data = []
all_product_urls = set()
processed_urls = set()
visited_pages = set()
print("🔄 Reset - změňte URL v Buňce 2")